In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta, timezone
import matplotlib as plot
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import itertools as it
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical
#from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import json

enc = LabelEncoder()

In [108]:
BTC_df = pd.read_csv('20250202-20170908_BTC-USDT_1D_okx_ohlc_M.csv')
BNB_df = pd.read_csv('20250127-20170907_BNB-USDT_1H_ohlc_M.csv')
DOGE_df = pd.read_csv('20250202-20170908_DOGE-USDT_1D_okx_ohlc_M.csv')
XRP_df = pd.read_csv('20250202-20170908_XRP-USDT_1D_okx_ohlc_M.csv')
#DOT_df = pd.read_csv('2025-01-27-2017-09-07_DOT-USDT_1H_ohlc_M.csv')
#SHIB_df = pd.read_csv('20250202-20170908_SHIB-USDT_1D_okx_ohlc_M.csv')
#df['Date'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %I:%M:%S %p')


In [161]:
#RSI CALC
def get_up_or_down(df, period):
    for i in range(len(df)):
        if i > 0:
            if df.iloc[i]['close'] >= df.iloc[i-1]['close']:
                df.at[i, 'gain_'+str(period)] = df.iloc[i]['close'] - df.iloc[i-1]['close']
                df.at[i, 'loss_'+str(period)] = 0
            elif df.iloc[i]['close'] < df.iloc[i-1]['close']:
                df.at[i, 'loss_'+str(period)] = df.iloc[i-1]['close'] - df.iloc[i]['close']
                df.at[i, 'gain_'+str(period)] = 0
            else:
                df.at[i, 'gain_'+str(period)] = 0
                df.at[i, 'loss_'+str(period)] = 0
    return df

def get_up_or_down_bin(df, offset):
    for i in range(len(df)):
        if i > 0:
            if df.iloc[i]['close'] >= df.iloc[i-offset]['close']:
                df.at[i, 'updown_'+str(offset)] = 1
            elif df.iloc[i]['close'] < df.iloc[i-offset]['close']:
                df.at[i, 'updown_'+str(offset)] = -1                
            else:
                df.at[i, 'updown_'+str(offset)] = 0
    return df
  
def get_relative_strength_index(df, period):
    try:
        df['Date'] = pd.to_datetime(df['timestamp'])
    except:
        df['Date'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
    #df['Date'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %I:%M:%S %p')
    #df['Date'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %I:%M:%S %p')
    #df['Date'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

    df.set_index(df['Date'])
    df = get_up_or_down(df, period)
    return df

def get_average_gains(df, period):
    for i in range(len(df)):
        n, up, down = 0, 0, 0
        if i == period:
            while n < period:
                if df.iloc[i-n]['gain_'+str(period)] > 0:
                    up += df.iloc[i-n]['gain_'+str(period)]
                elif df.iloc[i-n]['loss_'+str(period)] > 0:
                    down += df.iloc[i-n]['loss_'+str(period)]
                else:
                    up += 0
                    down += 0
                n += 1
            df.at[i, 'ag_'+str(period)] = up/period
            df.at[i, 'al_'+str(period)] = down/period
        elif i > period:
            df.at[i, 'ag_'+str(period)] = (df.iloc[i-1]['ag_'+str(period)] * (period - 1) + df.iloc[i]['gain_'+str(period)])/period
            df.at[i, 'al_'+str(period)] = (df.iloc[i-1]['al_'+str(period)] * (period - 1) + df.iloc[i]['loss_'+str(period)])/period
            df['ag_'+str(period)] = df['ag_'+str(period)].fillna(0)
            df['al_'+str(period)] = df['al_'+str(period)].fillna(0)
    return df

def get_relative_strength(df, period):
    df = get_relative_strength_index(df,period)
    df = get_average_gains(df, period)
    rs_col = f'rs_{period}' #added
    rsi_col = f'rsi_{period}'#added
    df[rs_col] = np.nan#added
    df[rsi_col] = np.nan    #added
    #for i in range(len(df)):
    '''
    if i >= period:
            df.at[i, 'rs_'+str(period)] = df.iloc[i]['ag_'+str(period)]/df.iloc[i]['al_'+str(period)]
            df.at[i, 'rsi_'+str(period)] = (100-(100/(1+df.iloc[i]['rs_'+str(period)])))
    '''
    for i in range(len(df)):
        if i >= period and df.iloc[i]['al_'+str(period)] != 0:
            df.at[i, rs_col] = df.iloc[i]['ag_'+str(period)] / df.iloc[i]['al_'+str(period)]
            df.at[i, rsi_col] = 100 - (100 / (1 + df.at[i, rs_col]))
        elif i >= period:
            df.at[i, rs_col] = 0
            df.at[i, rsi_col] = 100

    return df

##MONEY FLOW
def get_typical_price(high, low, close):
    typical_price = (high+low+close/3)
    return typical_price

def get_raw_money_flow(typical_price, volume):
    money_flow = typical_price * volume
    return money_flow

def get_money_flow_ratio(money_flow, window=14):
    signal = np.where(money_flow > money_flow.shift(1), 1, np.where(money_flow < money_flow.shift(1), -1, 0))
    money_flow_s = money_flow * signal
    
    money_flow_positive = money_flow_s.rolling(window).apply(lambda x: np.sum(np.where(x >= 0.0, x, 0.0)), raw=True)
    money_flow_negative = abs(money_flow_s.rolling(window).apply(lambda x: np.sum(np.where(x < 0.0, x, 0.0)), raw=True))
    
    money_flow_ratio = money_flow_positive / money_flow_negative
    
    return money_flow_ratio

def get_money_flow_index(money_flow_ratio):
    money_flow_index = 100. - 100./(1. + money_flow_ratio)
    return money_flow_index

def money_flow_index(high, low, close, volume, window=14):
    mfr = get_money_flow_ratio((high+low+close/3) * volume, window)
    mfi = 100. - 100./(1. + mfr)
    return mfi

#Choppiness index
def get_ci(high, low, close, lookback):
    tr1 = pd.DataFrame(high - low).rename(columns = {0:'tr1'})
    tr2 = pd.DataFrame(abs(high - close.shift(1))).rename(columns = {0:'tr2'})
    tr3 = pd.DataFrame(abs(low - close.shift(1))).rename(columns = {0:'tr3'})
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').dropna().max(axis = 1)
    atr = tr.rolling(1).mean()
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    ci = 100 * np.log10((atr.rolling(lookback).sum()) / (highh - lowl)) / np.log10(lookback)
    return ci

#Feature Extraction
def feature_extraction(df, time_steps_arr):
    df['range'] = df['high'] - df['low']
    df['range%'] = df['range']/df['close']
    df['obv'] = (np.sign(df['close'].diff()) * df['volume_ccy']).fillna(0).cumsum()    
    df['return'] = df['close'].pct_change() 
    
    features_list = ['range', 'range%', 'obv', 'return']
    for time in time_steps_arr:
        #df['return_'+str(time)] = (df.close / df.close.shift(time)) - 1
        df['return_'+str(time)] = (df['close'] / df['close'].shift(time)) - 1
        df = get_relative_strength(df, time)
        df = get_up_or_down_bin(df, time)
        df['std_'+str(time)] = df['return_'+str(time)].rolling(time).std()
        df['ma_'+str(time)] = df['close'].rolling(time).mean()
        df['mfi_'+str(time)]= money_flow_index(df['high'], df['low'], df['close'], df['volume_ccy'], time)
        df['ma_'+str(time)] = df['close'].rolling(time).mean()
        df['avgvolm_'+str(time)] = df['volume_ccy'].rolling(time).mean()
        df['avgvolty_'+str(time)] = df['std_'+str(time)].rolling(time).mean()
        df['rtrend_'+str(time)] = df['updown_'+str(time)].rolling(time).sum()
        df['ci_'+str(time)] = get_ci(df['high'], df['low'], df['close'], time)
        
        features_list.extend(['return_' + str(time), 'rs_' + str(time), 'updown_' + str(time),'std_' + str(time), 'ma_' + str(time), 'mfi_' + str(time),'avgvolm_' + str(time), 'avgvolty_' + str(time), 'rtrend_' + str(time),'ci_' + str(time)])
    return df, features_list

def convert_markov_states(df):
    '''
    for column in df.columns:
        for item in column:
            if item == "dtrending" or item == "down":
                item = 1                
            elif item == "rangebound" or item == "flat":
                item = 2
            elif item == "utrending" or item == "up":
                item = 3                
            else:
                item == 0
                
    '''
    for col_name, col_data in df.iteritems():
        col_data = col_data.str.strip().str.lower()
        for item in col_data:
            if str(item) == "dtrending" or item == "down":
                item = 1                
            elif item == "rangebound" or item == "flat":
                item = 2
            elif item == "utrending" or item == "up":
                item = 3                
            else:
                item == 0        
    return df
                
def training_states_shift(df, time_steps_arr):
    for time in time_steps_arr:
        df['mStateSt_'+str(time)] = df.m_st.shift(time)
        df['mStateMid_'+str(time)] = df.m_mt.shift(time)
        df['mStateLt_'+str(time)] = df.m_lt.shift(time)
        
        training_list = []
        training_list.append('mStateSt_'+str(time))
        training_list.append('mStateMid_'+str(time))
        training_list.append('mStateLt_'+str(time))
    return df, training_list

In [110]:
BTC_df.columns = BTC_df.columns.str.strip().str.lower()
XRP_df.columns = XRP_df.columns.str.strip().str.lower()
BNB_df.columns = BNB_df.columns.str.strip().str.lower()
DOGE_df.columns = DOGE_df.columns.str.strip().str.lower()

'''
BTC_df['markov_mt'] = BTC_df['m_mt'].astype('str') 
BTC_df['markov_mt'] = enc.fit_transform(BTC_df['m_mt'])

BTC_df['markov_st'] = BTC_df['m_st'].astype('str')
BTC_df['markov_st'] = enc.fit_transform(BTC_df['m_st'])

BTC_df['markov_lt'] = BTC_df['m_lt'].astype('str')
BTC_df['markov_lt'] = enc.fit_transform(BTC_df['m_lt'])


XRP_df['markov_mt'] = XRP_df['m_mt'].astype('str') 
XRP_df['markov_mt'] = enc.fit_transform(XRP_df['m_mt'])

XRP_df['markov_st'] = XRP_df['m_st'].astype('str')
XRP_df['markov_st'] = enc.fit_transform(XRP_df['m_st'])

XRP_df['markov_lt'] = XRP_df['m_lt'].astype('str')
XRP_df['markov_lt'] = enc.fit_transform(XRP_df['m_lt'])


BNB_df['markov_mt'] = BNB_df['m_mt'].astype('str') 
BNB_df['markov_mt'] = enc.fit_transform(BNB_df['m_mt'])

BNB_df['markov_st'] = BNB_df['m_st'].astype('str')
BNB_df['markov_st'] = enc.fit_transform(BNB_df['m_st'])

BNB_df['markov_lt'] = BNB_df['m_lt'].astype('str')
BNB_df['markov_lt'] = enc.fit_transform(BNB_df['m_lt'])


DOGE_df['markov_mt'] = DOGE_df['m_mt'].astype('str') 
DOGE_df['markov_mt'] = enc.fit_transform(DOGE_df['m_mt'])

DOGE_df['markov_st'] = DOGE_df['m_st'].astype('str')
DOGE_df['markov_st'] = enc.fit_transform(DOGE_df['m_st'])

DOGE_df['markov_lt'] = DOGE_df['m_lt'].astype('str')
DOGE_df['markov_lt'] = enc.fit_transform(DOGE_df['m_lt'])
'''

#time_periods = (2,3,4,5,6,7,8,9,10,12,15,17,20,25,30,35,40,45,50,60,70,80,90,100,150,200)
#time_periods = (2,3,5,7,10,14,19,25,35,50,100,200)
#time_periods = (2,15,50,100)
time_periods = (2,5,10,15,25,50,100)
#time_periods = (2,15)
features_list = []
BTC_df, features_list = feature_extraction(BTC_df, time_periods)
BTC_df, training_list = training_states_shift(BTC_df, time_periods)

XRP_df = feature_extraction(XRP_df, time_periods)[0]
XRP_df = training_states_shift(XRP_df, time_periods)[0]

BNB_df = feature_extraction(BNB_df, time_periods)[0]
BNB_df = training_states_shift(BNB_df, time_periods)[0]

DOGE_df = feature_extraction(DOGE_df, time_periods)[0]
DOGE_df = training_states_shift(DOGE_df, time_periods)[0]

#print(features_list)

'''
stack_df = BTC_df.append(XRP_df)
stack_df = stack_df.append(BNB_df)
stack_df = stack_df.append(DOGE_df)
'''

C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'loss_'+str(period)] = 0
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'ag_'+str(period)] = up/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'gain_'+str(period)] = 0
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'ag_'+str(period)] = up/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'ag_'+str(period)] = up/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'al_'+str(period)] = down/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'loss_'+str(period)] = 0
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'ag_'+str(period)] = up/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1701523453.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

'\nstack_df = BTC_df.append(XRP_df)\nstack_df = stack_df.append(BNB_df)\nstack_df = stack_df.append(DOGE_df)\n'

In [111]:
"""
next steps
-merge the columns/format for all training data
-label more data
-add in funding rates, implied vols, spx, gold, qqq, silver, copper, tbill rate, fed decision day, money supply (M2)
-stack the training data
-set this up as training, validation, prediction
-try LR, RF, and NN
-research how to identify rangebound markets' peaks and troughs
-figure out how to re-apply predicted markov states to be fed into training
-run this three times: try both 
    1) predict short term, then 2) mid term then 3) long term
    1) predict long term, then 2) mid term then 3) short term
    and see which predicts better
-apply Kelly Criterion
-research stop losses

"""
#print(list(XRP_df.columns))
#print(list(BTC_df.columns))
#print(features_list)
#print(type(features_list))
#print(type(training_list))

print(list(XRP_df.columns))
print(list(BTC_df.columns))
print(list(BNB_df.columns))
print(list(DOGE_df.columns))


['timestamp', 'open', 'high', 'low', 'close', 'volume', 'volume_ccy', 'volccyquote', 'm_mt', 'm_st', 'm_lt', 'm_mt_up', 'm_mt_down', 'm_mt_flat', 'm_st_up', 'm_st_down', 'm_st_flat', 'm_lt_up', 'm_lt_down', 'm_lt_flat', 'range', 'range%', 'obv', 'return', 'return_2', 'Date', 'loss_2', 'gain_2', 'ag_2', 'al_2', 'rs_2', 'rsi_2', 'updown_2', 'std_2', 'ma_2', 'mfi_2', 'avgvolm_2', 'avgvolty_2', 'rtrend_2', 'ci_2', 'return_5', 'loss_5', 'gain_5', 'ag_5', 'al_5', 'rs_5', 'rsi_5', 'updown_5', 'std_5', 'ma_5', 'mfi_5', 'avgvolm_5', 'avgvolty_5', 'rtrend_5', 'ci_5', 'return_10', 'loss_10', 'gain_10', 'ag_10', 'al_10', 'rs_10', 'rsi_10', 'updown_10', 'std_10', 'ma_10', 'mfi_10', 'avgvolm_10', 'avgvolty_10', 'rtrend_10', 'ci_10', 'return_15', 'loss_15', 'gain_15', 'ag_15', 'al_15', 'rs_15', 'rsi_15', 'updown_15', 'std_15', 'ma_15', 'mfi_15', 'avgvolm_15', 'avgvolty_15', 'rtrend_15', 'ci_15', 'return_25', 'loss_25', 'gain_25', 'ag_25', 'al_25', 'rs_25', 'rsi_25', 'updown_25', 'std_25', 'ma_25', 'm

In [112]:
all_cols = BTC_df.columns.tolist()
BTC_df = BTC_df.reset_index(drop=True)
XRP_df = XRP_df.reset_index(drop=True)
BNB_df = BNB_df.reset_index(drop=True)
DOGE_df = DOGE_df.reset_index(drop=True)
result = pd.concat([BTC_df, XRP_df, BNB_df, DOGE_df], axis=0)

In [164]:
#print(list(result.columns))
#print(type(features_list))
#print(features_list)
#print(dataCleaned_mtup.head())

str_to_int = {
    "utrending": 3,
    "rangebound": 2,
    "dtrending": 1,
    "up": 3,
    "flat": 2,
    "down": 1,    
}

def map_strings(x):
    return str_to_int.get(x, 0)

# Apply to all object (string) columns
for col in result.select_dtypes(include='object').columns:
    result[col] = result[col].apply(map_strings)

# If you want to include NaN values as 0:
result = result.fillna(0)
print(result.head(201))

     timestamp    open    high     low   close        volume    volume_ccy  \
0            0     1.0  4901.0     1.0  4901.0     19.260000  9.439326e+04   
1            0  4901.0  4999.0  4790.0  4989.0      0.580096  2.894099e+03   
2            0  4989.0  5922.0  4989.0  5741.6     20.739843  1.190799e+05   
3            0  5741.6  5849.9  5473.7  5849.9     19.259580  1.126666e+05   
4            0  5849.9  5849.9  5686.4  5848.0      8.658607  5.063553e+04   
..         ...     ...     ...     ...     ...           ...           ...   
196          0  9357.4  9758.3  8786.0  8963.0  11222.110840  1.047460e+08   
197          0  8963.1  9178.0  8652.9  8800.8   8610.270343  7.624737e+07   
198          0  8817.6  9386.8  8780.0  9236.6   7446.504090  6.853483e+07   
199          0  9238.0  9429.9  8866.2  9357.7   7870.328640  7.223183e+07   
200          0  9357.7  9566.0  9157.3  9299.7   8734.322895  8.185606e+07   

      volccyquote  m_mt  m_st  ...  mStateLt_15  mStateSt_25  m

In [165]:
#m_mt up
features_list.extend(['m_mt', 'm_st', 'm_lt'])

features_list_mtup = features_list.copy()
features_list_mtup.extend(training_list)
features_list_mtup.append('m_mt_up')
drop_list_mtup = [item for item in all_cols if item not in features_list_mtup]

dataCleaned_mtup = result.drop(drop_list_mtup, axis=1)
dataCleaned_mtup = dataCleaned_mtup.dropna()

y_mtup = dataCleaned_mtup['m_mt_up'] # Target variable
X_mtup = dataCleaned_mtup.drop('m_mt_up', axis=1)


In [166]:
#m_mt flat
features_list_mtfl = features_list.copy()
features_list_mtfl.extend(training_list)
features_list_mtfl.append('m_mt_flat')
drop_list_mtfl = [item for item in all_cols if item not in features_list_mtfl]

dataCleaned_mtfl = result.drop(drop_list_mtfl, axis=1)
dataCleaned_mtfl = dataCleaned_mtfl.dropna()

y_mtfl = dataCleaned_mtfl['m_mt_flat'] # Target variable
X_mtfl = dataCleaned_mtfl.drop('m_mt_flat', axis=1)

In [167]:
#m_mt down
features_list_mtdn = features_list.copy()
features_list_mtdn.extend(training_list)
features_list_mtdn.append('m_mt_down')
drop_list_mtdn = [item for item in all_cols if item not in features_list_mtdn]

dataCleaned_mtdn = result.drop(drop_list_mtdn, axis=1)
dataCleaned_mtdn = dataCleaned_mtdn.dropna()

y_mtdn = dataCleaned_mtdn['m_mt_down'] # Target variable
X_mtdn = dataCleaned_mtdn.drop('m_mt_down', axis=1)

In [168]:
X_train_mtup, X_test_mtup, y_train_mtup, y_test_mtup = train_test_split(X_mtup, y_mtup, test_size=0.3, random_state=42)
X_train_mtdn, X_test_mtdn, y_train_mtdn, y_test_mtdn = train_test_split(X_mtdn, y_mtdn, test_size=0.3, random_state=42)
X_train_mtfl, X_test_mtfl, y_train_mtfl, y_test_mtfl = train_test_split(X_mtfl, y_mtfl, test_size=0.3, random_state=42)
model_mtup = LogisticRegression(random_state=42, max_iter=99999) # Initialize the model
model_mtdn = LogisticRegression(random_state=42, max_iter=99999) # Initialize the model
model_mtfl = LogisticRegression(random_state=42, max_iter=99999) # Initialize the model
model_mtup.fit(X_train_mtup, y_train_mtup) # Train the model
model_mtdn.fit(X_train_mtdn, y_train_mtdn) # Train the model
model_mtfl.fit(X_train_mtfl, y_train_mtfl) # Train the model

LogisticRegression(max_iter=99999, random_state=42)

In [169]:
print(list(dataCleaned.columns))


['markov_mt', 'range', 'range%', 'obv', 'return', 'return_2', 'rs_2', 'updown_2', 'std_2', 'ma_2', 'mfi_2', 'avgvolm_2', 'avgvolty_2', 'rtrend_2', 'ci_2', 'return_15', 'rs_15', 'updown_15', 'std_15', 'ma_15', 'mfi_15', 'avgvolm_15', 'avgvolty_15', 'rtrend_15', 'ci_15', 'mStateSt_15', 'mStateMid_15', 'mStateLt_15', 'volCcyQuote']


In [170]:
#classification algorithms 
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
"""
y_pred_mtup = model_mtup.predict(X_test_mtup)
y_pred_mtdn = model_mtdn.predict(X_test_mtdn)
y_pred_mtfl = model_mtfl.predict(X_test_mtfl)

print("Accuracy MT UP:", accuracy_score(y_test_mtup, y_pred_mtup))
print(classification_report(y_test_mtup, y_pred_mtup))

print("Accuracy MT DOWN:", accuracy_score(y_test_mtdn, y_pred_mtdn))
print(classification_report(y_test_mtdn, y_pred_mtdn))

print("Accuracy MT FLAT:", accuracy_score(y_test_mtfl, y_pred_mtfl))
print(classification_report(y_test_mtfl, y_pred_mtfl))

Accuracy MT UP: 0.8200025192089684
              precision    recall  f1-score   support

         0.0       0.94      0.86      0.90      7306
         1.0       0.20      0.41      0.27       633

    accuracy                           0.82      7939
   macro avg       0.57      0.63      0.58      7939
weighted avg       0.88      0.82      0.85      7939

Accuracy MT DOWN: 0.8194986774152916
              precision    recall  f1-score   support

         0.0       0.90      0.89      0.90      7018
         1.0       0.25      0.27      0.26       921

    accuracy                           0.82      7939
   macro avg       0.57      0.58      0.58      7939
weighted avg       0.83      0.82      0.82      7939

Accuracy MT FLAT: 0.8589242977705001
              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92      6956
         1.0       0.43      0.41      0.42       983

    accuracy                           0.86      7939
   macro avg       

In [171]:
DOT_df = pd.read_csv('2025-01-27-2017-09-07_DOT-USDT_1H_ohlc_M.csv')

DOT_df.columns = DOT_df.columns.str.strip().str.lower()

time_periods = (2,5,10,15,25,50,100)

features_list_dot = []
DOT_df, features_list_dot = feature_extraction(DOT_df, time_periods)

str_to_int = {
    "utrending": 3,
    "rangebound": 2,
    "dtrending": 1,
    "up": 3,
    "flat": 2,
    "down": 1,    
}

def map_strings(x):
    return str_to_int.get(x, 0)

# Apply to all object (string) columns
for col in DOT_df.select_dtypes(include='object').columns:
    DOT_df[col] = DOT_df[col].apply(map_strings)

# If you want to include NaN values as 0:
DOT_df = DOT_df.fillna(0)
print(DOT_df.head(201))

#m_mt up
features_list_dot.extend(['m_mt', 'm_st', 'm_lt'])

features_list_mtup = features_list.copy()
features_list_mtup.extend(training_list)
features_list_mtup.append('m_mt_up')
drop_list_mtup = [item for item in all_cols if item not in features_list_mtup]

dataCleaned_mtup = DOT_df.drop(drop_list_mtup, axis=1)
dataCleaned_mtup = dataCleaned_mtup.dropna()

y_mtup = dataCleaned_mtup['m_mt_up'] # Target variable
X_mtup = dataCleaned_mtup.drop('m_mt_up', axis=1)

#m_mt flat
features_list_mtfl = features_list.copy()
features_list_mtfl.extend(training_list)
features_list_mtfl.append('m_mt_flat')
drop_list_mtfl = [item for item in all_cols if item not in features_list_mtfl]

dataCleaned_mtfl = DOT_df.drop(drop_list_mtfl, axis=1)
dataCleaned_mtfl = dataCleaned_mtfl.dropna()

y_mtfl = dataCleaned_mtfl['m_mt_flat'] # Target variable
X_mtfl = dataCleaned_mtfl.drop('m_mt_flat', axis=1)

#m_mt down
features_list_mtdn = features_list.copy()
features_list_mtdn.extend(training_list)
features_list_mtdn.append('m_mt_down')
drop_list_mtdn = [item for item in all_cols if item not in features_list_mtdn]

dataCleaned_mtdn = DOT_df.drop(drop_list_mtdn, axis=1)
dataCleaned_mtdn = dataCleaned_mtdn.dropna()

y_mtdn = dataCleaned_mtdn['m_mt_down'] # Target variable
X_mtdn = dataCleaned_mtdn.drop('m_mt_down', axis=1)


C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'loss_'+str(period)] = 0
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, 'ag_'+str(period)] = up/period
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

     timestamp    open     high     low   close       volume   volume_ccy  \
0            0  2.5000  10.0000  2.5000  3.7800  563244.8890  2119031.467   
1            0  3.7859   3.8700  3.6055  3.8314  386915.6953  1445705.359   
2            0  3.8310   3.8500  3.4601  3.6800  526700.5637  1898993.308   
3            0  3.6709   4.0700  3.6368  3.8240  671441.0959  2596874.480   
4            0  3.8049   3.9500  3.5802  3.8300  854647.4708  3260625.580   
..         ...     ...      ...     ...     ...          ...          ...   
196          0  5.9000   5.9374  5.7000  5.8298  825980.8914  4798303.945   
197          0  5.8256   5.8851  5.7834  5.8079  489102.8176  2857223.020   
198          0  5.8035   5.9284  5.7605  5.8954  423732.1717  2489875.764   
199          0  5.8895   6.0475  5.8894  6.0098  418453.2806  2508701.179   
200          0  6.0177   6.1030  5.9946  6.1028  234018.2803  1414737.353   

     volccyquote  m_mt  m_st  ...    rs_100    rsi_100  updown_100   std_10

C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:142: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['mfi_'+str(time)]= money_flow_index(df['high'], df['low'], df['close'], df['volume_ccy'], time)
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['avgvolm_'+str(time)] = df['volume_ccy'].rolling(time).mean()
C:\Users\Luke\AppData\Local\Temp\ipykernel_14068\1716631908.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [176]:
features_list_mtup = features_list.copy()
features_list_mtup.extend(training_list)
features_list_mtup.append('m_mt_up')
drop_list_mtup = [item for item in all_cols if item not in features_list_mtup]

#dataCleaned_mtup = DOT_df.drop(drop_list_mtup, axis=1)
dataCleaned_mtup = dataCleaned_mtup.dropna()

y_mtup = dataCleaned_mtup['m_mt_up'] # Target variable
X_mtup = dataCleaned_mtup.drop('m_mt_up', axis=1)

#m_mt flat
features_list_mtfl = features_list.copy()
features_list_mtfl.extend(training_list)
features_list_mtfl.append('m_mt_flat')
drop_list_mtfl = [item for item in all_cols if item not in features_list_mtfl]

#dataCleaned_mtfl = DOT_df.drop(drop_list_mtfl, axis=1)
dataCleaned_mtfl = dataCleaned_mtfl.dropna()

y_mtfl = dataCleaned_mtfl['m_mt_flat'] # Target variable
X_mtfl = dataCleaned_mtfl.drop('m_mt_flat', axis=1)

#m_mt down
features_list_mtdn = features_list.copy()
features_list_mtdn.extend(training_list)
features_list_mtdn.append('m_mt_down')
drop_list_mtdn = [item for item in all_cols if item not in features_list_mtdn]

#dataCleaned_mtdn = DOT_df.drop(drop_list_mtdn, axis=1)
dataCleaned_mtdn = dataCleaned_mtdn.dropna()

y_mtdn = dataCleaned_mtdn['m_mt_down'] # Target variable
X_mtdn = dataCleaned_mtdn.drop('m_mt_down', axis=1)

In [178]:

y_pred_mtup = model_mtup.predict(X_mtup)
y_pred_mtdn = model_mtdn.predict(X_mtdn)
y_pred_mtfl = model_mtfl.predict(X_mtfl)

print("Accuracy MT UP:", accuracy_score(y_mtup, y_pred_mtup))
print(classification_report(y_mtup, y_pred_mtup))

print("Accuracy MT DOWN:", accuracy_score(y_mtdn, y_pred_mtdn))
print(classification_report(y_mtdn, y_pred_mtdn))

print("Accuracy MT FLAT:", accuracy_score(y_mtfl, y_pred_mtfl))
print(classification_report(y_mtfl, y_pred_mtfl))

Accuracy MT UP: 0.8173916329692755
              precision    recall  f1-score   support

         0.0       0.94      0.85      0.90     24315
         1.0       0.20      0.40      0.26      2146

    accuracy                           0.82     26461
   macro avg       0.57      0.63      0.58     26461
weighted avg       0.88      0.82      0.84     26461

Accuracy MT DOWN: 0.8218132345716337
              precision    recall  f1-score   support

         0.0       0.90      0.89      0.90     23460
         1.0       0.24      0.26      0.25      3001

    accuracy                           0.82     26461
   macro avg       0.57      0.58      0.57     26461
weighted avg       0.83      0.82      0.83     26461

Accuracy MT FLAT: 0.8569970900570651
              precision    recall  f1-score   support

         0.0       0.91      0.92      0.92     23083
         1.0       0.44      0.40      0.42      3378

    accuracy                           0.86     26461
   macro avg       